#  Kriz Tahmin - NLP Model Karşılaştırması

## Amaç
1. **7 NLP Modeli** (ALBERT, BERT, RoBERTa, USE, Word2Vec, GloVe, FastText) ile **AYLIK** kriz skoru hesapla
2. **Enflasyon** ile korelasyon analizi yap
3. **Kelime Sayma** yöntemi ile karşılaştır
4. En iyi tahmin modelini bul

## Yenilik
- Önceki analizde NLP modelleri sadece **yıllık ortalama** skor veriyordu
- Bu notebook'ta **AYLIK** skorlar hesaplanıyor → Zaman serisi analizi mümkün!

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# Kütüphaneler
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
import warnings
warnings.filterwarnings('ignore')

# NLP Kütüphaneleri - Yükle
!pip install sentence-transformers -q
!pip install tensorflow-hub -q
!pip install gensim -q
!pip install fasttext -q

from sentence_transformers import SentenceTransformer, util
import torch
import tensorflow_hub as hub
import gensim.downloader as api
import fasttext

plt.rcParams['font.family'] = 'DejaVu Sans'

SONUC_DIZINI = '/content/drive/MyDrive/Colab Notebooks/Sonuclar/nlp'
DATA_DIZINI = '/content/drive/MyDrive/Colab Notebooks/DataFrames'
MODEL_DIZINI = '/content/drive/MyDrive/Colab Notebooks/NLP Modeller'

print(' Kütüphaneler yüklendi!')

##  Bölüm 1: Veri Yükleme

In [ ]:
# İşlenmiş haber verisini yükle (İngilizce + Lemmatized)
csv_path = f'{DATA_DIZINI}/BorsaHaberleri_Neutr_2018-01-01_2018-12-31-ingilizce-KokleriBulunmus.csv'
df = pd.read_csv(csv_path)

# Tarih sütununu işle (sütun adı 'Tarih')
df['date'] = pd.to_datetime(df['Tarih'], format='%a, %d %b %Y %H:%M:%S GMT', errors='coerce')
df['month'] = df['date'].dt.month
df = df.dropna(subset=['lemmatized_text', 'date'])

print(f' {len(df)} haber yüklendi!')
print(f' Sütunlar: {df.columns.tolist()}')
print(f'\n Aylık dağılım:')
print(df['month'].value_counts().sort_index())

In [ ]:
# Türkçe ham haberleri de yükle (kelime sayma için)
csv_path_tr = f'{DATA_DIZINI}/BorsaHaberleri_Neutr_2018-01-01_2018-12-31.csv'
df_tr = pd.read_csv(csv_path_tr)
df_tr['date'] = pd.to_datetime(df_tr['Tarih'], format='%a, %d %b %Y %H:%M:%S GMT', errors='coerce')
df_tr = df_tr.dropna(subset=['date'])
df_tr['month'] = df_tr['date'].dt.month
print(f' {len(df_tr)} Türkçe haber yüklendi!')

In [ ]:
# 2018 Enflasyon verisi
enflasyon_2018 = {
    1: 10.35, 2: 10.26, 3: 10.23, 4: 10.85, 5: 12.15, 6: 15.39,
    7: 15.85, 8: 17.90, 9: 24.52, 10: 25.24, 11: 21.62, 12: 20.30
}

ay_adi = {1:'Oca', 2:'Şub', 3:'Mar', 4:'Nis', 5:'May', 6:'Haz',
          7:'Tem', 8:'Ağu', 9:'Eyl', 10:'Eki', 11:'Kas', 12:'Ara'}

HEDEF_CUMLE = 'Economic Crisis'
print(' Referans verileri yüklendi!')

## 🤖 Bölüm 2: NLP Modelleri ile AYLIK Skor Hesaplama

In [ ]:
# Yardımcı fonksiyonlar
def cosine_similarity_np(v1, v2):
    norm1 = np.linalg.norm(v1)
    norm2 = np.linalg.norm(v2)
    if norm1 > 0 and norm2 > 0:
        return np.dot(v1, v2) / (norm1 * norm2)
    return 0.0

def get_average_vector(sentence, model, vector_size=300):
    words = str(sentence).lower().split()
    vectors = [model[word] for word in words if word in model.key_to_index]
    if vectors:
        return np.mean(vectors, axis=0)
    return np.zeros(vector_size)

aylik_nlp_sonuclari = {}
print(' Yardımcı fonksiyonlar tanımlandı!')

### 2.1 ALBERT Modeli

In [ ]:
print('🤖 ALBERT modeli yükleniyor...')
albert_model = SentenceTransformer('albert-base-v2')
hedef_embedding = albert_model.encode(HEDEF_CUMLE, convert_to_tensor=True)

albert_aylik = []
for month in range(1, 13):
    ay_haberleri = df[df['month'] == month]['lemmatized_text'].tolist()
    if len(ay_haberleri) > 0:
        corpus_embeddings = albert_model.encode(ay_haberleri, convert_to_tensor=True)
        cosine_scores = util.pytorch_cos_sim(hedef_embedding, corpus_embeddings)
        ortalama_skor = torch.mean(cosine_scores).item()
    else:
        ortalama_skor = 0
    albert_aylik.append({'month': month, 'ay': ay_adi[month], 'haber_sayisi': len(ay_haberleri), 'kriz_skor': ortalama_skor})
    print(f'  {ay_adi[month]}: {len(ay_haberleri)} haber, Kriz Skoru: {ortalama_skor:.4f}')

aylik_nlp_sonuclari['ALBERT'] = pd.DataFrame(albert_aylik)
print(' ALBERT tamamlandı!')

### 2.2 BERT Modeli

In [ ]:
print('🤖 BERT modeli yükleniyor...')
bert_model = SentenceTransformer('bert-base-nli-mean-tokens')
hedef_embedding = bert_model.encode(HEDEF_CUMLE, convert_to_tensor=True)

bert_aylik = []
for month in range(1, 13):
    ay_haberleri = df[df['month'] == month]['lemmatized_text'].tolist()
    if len(ay_haberleri) > 0:
        corpus_embeddings = bert_model.encode(ay_haberleri, convert_to_tensor=True)
        cosine_scores = util.pytorch_cos_sim(hedef_embedding, corpus_embeddings)
        ortalama_skor = torch.mean(cosine_scores).item()
    else:
        ortalama_skor = 0
    bert_aylik.append({'month': month, 'ay': ay_adi[month], 'haber_sayisi': len(ay_haberleri), 'kriz_skor': ortalama_skor})
    print(f'  {ay_adi[month]}: Kriz Skoru: {ortalama_skor:.4f}')

aylik_nlp_sonuclari['BERT'] = pd.DataFrame(bert_aylik)
print(' BERT tamamlandı!')

### 2.3 RoBERTa Modeli

In [ ]:
print('🤖 RoBERTa modeli yükleniyor...')
roberta_model = SentenceTransformer('roberta-base-nli-mean-tokens')
hedef_embedding = roberta_model.encode(HEDEF_CUMLE, convert_to_tensor=True)

roberta_aylik = []
for month in range(1, 13):
    ay_haberleri = df[df['month'] == month]['lemmatized_text'].tolist()
    if len(ay_haberleri) > 0:
        corpus_embeddings = roberta_model.encode(ay_haberleri, convert_to_tensor=True)
        cosine_scores = util.pytorch_cos_sim(hedef_embedding, corpus_embeddings)
        ortalama_skor = torch.mean(cosine_scores).item()
    else:
        ortalama_skor = 0
    roberta_aylik.append({'month': month, 'ay': ay_adi[month], 'haber_sayisi': len(ay_haberleri), 'kriz_skor': ortalama_skor})
    print(f'  {ay_adi[month]}: Kriz Skoru: {ortalama_skor:.4f}')

aylik_nlp_sonuclari['RoBERTa'] = pd.DataFrame(roberta_aylik)
print(' RoBERTa tamamlandı!')

### 2.4 Universal Sentence Encoder (USE)

In [ ]:
print('🤖 USE modeli yükleniyor...')
try:
    use_model_path = f'{MODEL_DIZINI}/USE Files'
    use_model = hub.load(use_model_path)
    print('  Drive\'dan yüklendi!')
except Exception as e:
    print(f'  Drive yüklenemedi: {e}')
    print('  TF Hub\'dan indiriliyor...')
    use_model = hub.load('https://tfhub.dev/google/universal-sentence-encoder/4')

hedef_embedding_use = use_model([HEDEF_CUMLE])[0].numpy()

use_aylik = []
for month in range(1, 13):
    ay_haberleri = df[df['month'] == month]['lemmatized_text'].tolist()
    if len(ay_haberleri) > 0:
        corpus_embeddings = use_model(ay_haberleri).numpy()
        similarities = [cosine_similarity_np(hedef_embedding_use, emb) for emb in corpus_embeddings]
        ortalama_skor = np.mean(similarities)
    else:
        ortalama_skor = 0
    use_aylik.append({'month': month, 'ay': ay_adi[month], 'haber_sayisi': len(ay_haberleri), 'kriz_skor': ortalama_skor})
    print(f'  {ay_adi[month]}: Kriz Skoru: {ortalama_skor:.4f}')

aylik_nlp_sonuclari['USE'] = pd.DataFrame(use_aylik)
print(' USE tamamlandı!')

### 2.5 Word2Vec Modeli

In [ ]:
print('🤖 Word2Vec modeli yükleniyor...')
from gensim.models import KeyedVectors
import os

w2v_drive_path = f'{MODEL_DIZINI}/Word2Vec/word2vec-google-news-300.kv'
w2v_folder = f'{MODEL_DIZINI}/Word2Vec'

# Klasör yoksa oluştur
os.makedirs(w2v_folder, exist_ok=True)

try:
    # Önce Drive'dan yükle
    w2v_model = KeyedVectors.load(w2v_drive_path)
    print(f'   Drive\'dan yüklendi!')
except:
    print('  Drive\'da bulunamadı, indiriliyor...')
    print('  ⏳ word2vec-google-news-300 indiriliyor (~1.7GB)...')
    w2v_model = api.load('word2vec-google-news-300')
    print('   Drive\'a kaydediliyor...')
    w2v_model.save(w2v_drive_path)
    print(f'   Kaydedildi: {w2v_drive_path}')

vector_size_w2v = w2v_model.vector_size
print(f'  Vektör boyutu: {vector_size_w2v}')

hedef_vec_w2v = get_average_vector(HEDEF_CUMLE, w2v_model, vector_size_w2v)

w2v_aylik = []
for month in range(1, 13):
    ay_haberleri = df[df['month'] == month]['lemmatized_text'].tolist()
    if len(ay_haberleri) > 0:
        corpus_vecs = [get_average_vector(text, w2v_model, vector_size_w2v) for text in ay_haberleri]
        similarities = [cosine_similarity_np(hedef_vec_w2v, vec) for vec in corpus_vecs]
        ortalama_skor = np.mean(similarities)
    else:
        ortalama_skor = 0
    w2v_aylik.append({'month': month, 'ay': ay_adi[month], 'haber_sayisi': len(ay_haberleri), 'kriz_skor': ortalama_skor})
    print(f'  {ay_adi[month]}: Kriz Skoru: {ortalama_skor:.4f}')

aylik_nlp_sonuclari['Word2Vec'] = pd.DataFrame(w2v_aylik)
print(' Word2Vec tamamlandı!')

### 2.6 GloVe Modeli

In [ ]:
print('🤖 GloVe modeli yükleniyor...')
from gensim.models import KeyedVectors
try:
    glove_path = f'{MODEL_DIZINI}/GloVe File/glove.6B.300d.txt'
    glove_model = KeyedVectors.load_word2vec_format(glove_path, binary=False, no_header=True)
    vector_size_glove = 300
    print(f'  Drive\'dan yüklendi! Vektör boyutu: {vector_size_glove}')
except Exception as e:
    print(f'  Drive\'dan yüklenemedi: {e}')
    print('  glove-wiki-gigaword-100 indiriliyor...')
    glove_model = api.load('glove-wiki-gigaword-100')
    vector_size_glove = glove_model.vector_size

hedef_vec_glove = get_average_vector(HEDEF_CUMLE, glove_model, vector_size_glove)

glove_aylik = []
for month in range(1, 13):
    ay_haberleri = df[df['month'] == month]['lemmatized_text'].tolist()
    if len(ay_haberleri) > 0:
        corpus_vecs = [get_average_vector(text, glove_model, vector_size_glove) for text in ay_haberleri]
        similarities = [cosine_similarity_np(hedef_vec_glove, vec) for vec in corpus_vecs]
        ortalama_skor = np.mean(similarities)
    else:
        ortalama_skor = 0
    glove_aylik.append({'month': month, 'ay': ay_adi[month], 'haber_sayisi': len(ay_haberleri), 'kriz_skor': ortalama_skor})
    print(f'  {ay_adi[month]}: Kriz Skoru: {ortalama_skor:.4f}')

aylik_nlp_sonuclari['GloVe'] = pd.DataFrame(glove_aylik)
print(' GloVe tamamlandı!')

### 2.7 FastText Modeli

In [ ]:
print('🤖 FastText modeli yükleniyor...')
try:
    ft_path = f'{MODEL_DIZINI}/FastText Files/cc.en.300.bin'
    ft_model = fasttext.load_model(ft_path)
    print(f'  Drive\'dan yüklendi!')

    hedef_vec_ft = ft_model.get_sentence_vector(HEDEF_CUMLE.lower())

    ft_aylik = []
    for month in range(1, 13):
        ay_haberleri = df[df['month'] == month]['lemmatized_text'].tolist()
        if len(ay_haberleri) > 0:
            corpus_vecs = [ft_model.get_sentence_vector(str(text).lower()) for text in ay_haberleri]
            similarities = [cosine_similarity_np(hedef_vec_ft, vec) for vec in corpus_vecs]
            ortalama_skor = np.mean(similarities)
        else:
            ortalama_skor = 0
        ft_aylik.append({'month': month, 'ay': ay_adi[month], 'haber_sayisi': len(ay_haberleri), 'kriz_skor': ortalama_skor})
        print(f'  {ay_adi[month]}: Kriz Skoru: {ortalama_skor:.4f}')

    aylik_nlp_sonuclari['FastText'] = pd.DataFrame(ft_aylik)
    print(' FastText tamamlandı!')
except Exception as e:
    print(f' FastText atlandı: {e}')

### 2.8 Kelime Sayma Yöntemi (Baseline)

In [ ]:
print(' Kelime Sayma yöntemi...')
kriz_kelimeler = ['kriz', 'çöküş', 'düşüş', 'kayıp', 'zarar', 'dolar', 'euro', 'kur',
                  'devalüasyon', 'enflasyon', 'faiz', 'panik', 'korku', 'risk', 'belirsizlik']

kelime_aylik = []
for month in range(1, 13):
    ay_haberleri = df_tr[df_tr['month'] == month]
    texts = ' '.join(ay_haberleri['Başlık'].dropna().astype(str)).lower()
    words = texts.split()
    total = max(len(words), 1)
    kriz_count = sum(1 for w in words if any(k in w for k in kriz_kelimeler))
    kriz_pct = kriz_count / total * 100
    kelime_aylik.append({'month': month, 'ay': ay_adi[month], 'haber_sayisi': len(ay_haberleri), 'kriz_skor': kriz_pct})
    print(f'  {ay_adi[month]}: Kriz Skoru: {kriz_pct:.2f}%')

aylik_nlp_sonuclari['Kelime_Sayma'] = pd.DataFrame(kelime_aylik)
print(' Kelime Sayma tamamlandı!')

##  Bölüm 3: Sonuçları Birleştir ve Normalize Et

In [ ]:
# Tüm sonuçları birleştir
df_karsilastirma = pd.DataFrame({
    'month': range(1, 13),
    'ay': [ay_adi[m] for m in range(1, 13)],
    'enflasyon': [enflasyon_2018[m] for m in range(1, 13)]
})

for model_adi, df_model in aylik_nlp_sonuclari.items():
    df_karsilastirma[model_adi] = df_model['kriz_skor'].values

print(' Tüm Aylık Skorlar:')
print(df_karsilastirma.to_string(index=False))

In [ ]:
# Normalize et
df_norm = df_karsilastirma.copy()
df_norm['enflasyon_norm'] = (df_norm['enflasyon'] - df_norm['enflasyon'].min()) / (df_norm['enflasyon'].max() - df_norm['enflasyon'].min())

model_sutunlari = [col for col in df_karsilastirma.columns if col not in ['month', 'ay', 'enflasyon']]
for col in model_sutunlari:
    min_val = df_norm[col].min()
    max_val = df_norm[col].max()
    if max_val > min_val:
        df_norm[f'{col}_norm'] = (df_norm[col] - min_val) / (max_val - min_val)
    else:
        df_norm[f'{col}_norm'] = 0
print(' Normalizasyon tamamlandı!')

##  Bölüm 4: Korelasyon Analizi

In [ ]:
print('='*70)
print(' ENFLASYON İLE KORELASYON ANALİZİ')
print('='*70)

korelasyon_sonuclari = []
for model_adi in model_sutunlari:
    print(f'\n {model_adi}:')
    for lag in range(-2, 4):
        if lag == 0:
            skor = df_norm[f'{model_adi}_norm']
        else:
            skor = df_norm[f'{model_adi}_norm'].shift(lag)
        mask = ~skor.isna()
        if mask.sum() >= 5:
            r, p = stats.pearsonr(skor[mask], df_norm['enflasyon_norm'][mask])
            anlamli = '' if p < 0.05 else ''
            print(f'   Lag {lag:+d}: r={r:.4f}, p={p:.4f} {anlamli}')
            korelasyon_sonuclari.append({'Model': model_adi, 'Lag': lag, 'r': r, 'p': p, 'Anlamli': p < 0.05})

df_korelasyon = pd.DataFrame(korelasyon_sonuclari)

In [ ]:
print('\n' + '='*70)
print(' EN İYİ KORELASYONLAR (p < 0.05)')
print('='*70)

df_anlamli = df_korelasyon[df_korelasyon['Anlamli'] == True].copy()
df_anlamli['r_abs'] = df_anlamli['r'].abs()
df_anlamli = df_anlamli.sort_values('r_abs', ascending=False)
print(df_anlamli[['Model', 'Lag', 'r', 'p']].head(10).to_string(index=False))

if len(df_anlamli) > 0:
    en_iyi = df_anlamli.iloc[0]
    print(f'\n EN İYİ MODEL: {en_iyi["Model"]} (Lag={en_iyi["Lag"]}, r={en_iyi["r"]:.4f})')

##  Bölüm 5: Görselleştirmeler

In [ ]:
# Klasör yoksa oluştur
import os
os.makedirs(SONUC_DIZINI, exist_ok=True)

# 2x2 Grafik
fig, axes = plt.subplots(2, 2, figsize=(16, 12))

# Grafik 1: Tüm modeller
ax1 = axes[0, 0]
colors = plt.cm.tab10(np.linspace(0, 1, len(model_sutunlari)))
for i, model_adi in enumerate(model_sutunlari):
    ax1.plot(range(12), df_norm[f'{model_adi}_norm'], '-o', label=model_adi, color=colors[i], alpha=0.7, markersize=6)
ax1.plot(range(12), df_norm['enflasyon_norm'], 'k-s', label='Enflasyon', linewidth=3, markersize=8)
ax1.axvspan(7, 9, alpha=0.2, color='red')
ax1.set_xticks(range(12))
ax1.set_xticklabels(list(ay_adi.values()))
ax1.set_title('Tüm NLP Modelleri vs Enflasyon (2018)', fontweight='bold')
ax1.legend(loc='upper left', fontsize=8)
ax1.grid(True, alpha=0.3)

# Grafik 2: En iyi model
ax2 = axes[0, 1]
if len(df_anlamli) > 0:
    en_iyi_model = df_anlamli.iloc[0]['Model']
    ax2.plot(range(12), df_norm[f'{en_iyi_model}_norm'], 'b-o', label=en_iyi_model, linewidth=2, markersize=8)
    ax2.plot(range(12), df_norm['enflasyon_norm'], 'r-s', label='Enflasyon', linewidth=2, markersize=8)
    ax2.axvspan(7, 9, alpha=0.2, color='red')
    ax2.set_xticks(range(12))
    ax2.set_xticklabels(list(ay_adi.values()))
    ax2.set_title(f'En İyi: {en_iyi_model} (r={df_anlamli.iloc[0]["r"]:.4f})', fontweight='bold')
    ax2.legend()
    ax2.grid(True, alpha=0.3)

# Grafik 3: Korelasyon bar
ax3 = axes[1, 0]
en_iyi_r = df_korelasyon.loc[df_korelasyon.groupby('Model')['r'].idxmax()]
en_iyi_r = en_iyi_r.sort_values('r', ascending=True)
colors_bar = ['green' if r > 0.5 else 'orange' if r > 0.3 else 'red' for r in en_iyi_r['r']]
ax3.barh(en_iyi_r['Model'], en_iyi_r['r'], color=colors_bar, alpha=0.7)
ax3.axvline(x=0.5, color='green', linestyle='--')
ax3.set_xlabel('Pearson Korelasyon (r)')
ax3.set_title('Model Korelasyon Karşılaştırması', fontweight='bold')
ax3.grid(True, alpha=0.3, axis='x')

# Grafik 4: Heatmap
ax4 = axes[1, 1]
pivot = df_korelasyon.pivot(index='Model', columns='Lag', values='r')
sns.heatmap(pivot, annot=True, fmt='.2f', cmap='RdYlGn', center=0, ax=ax4)
ax4.set_title('Lag Analizi Heatmap', fontweight='bold')

plt.tight_layout()
plt.savefig(f'{SONUC_DIZINI}/nlp_model_karsilastirma.png', dpi=150, bbox_inches='tight')
plt.show()
print(' Grafik kaydedildi!')

##  Bölüm 6: Özet Rapor

In [ ]:
print('='*70)
print('  NLP MODEL KARŞILAŞTIRMA RAPORU')
print('='*70)
print(f'\n ANALİZ ÖZETİ:')
print(f'   • Toplam haber: {len(df)}')
print(f'   • Dönem: Ocak - Aralık 2018')
print(f'   • Hedef ifade: "{HEDEF_CUMLE}"')
print(f'   • Test edilen model: {len(model_sutunlari)}')

print('\n MODEL SONUÇLARI:')
for model in model_sutunlari:
    model_df = df_korelasyon[df_korelasyon['Model'] == model]
    en_iyi = model_df.loc[model_df['r'].idxmax()]
    sonuc = '' if en_iyi['p'] < 0.05 else ''
    print(f'   {model:<15} Lag={int(en_iyi["Lag"]):+d}  r={en_iyi["r"]:.4f}  p={en_iyi["p"]:.4f} {sonuc}')

print('\n YORUM: Pozitif lag = NLP skoru enflasyonu ÖNCEden tahmin ediyor')

In [ ]:
# Excel kaydet
output_path = f'{SONUC_DIZINI}/9_NLP_Model_Karsilastirma_Sonuclari.xlsx'
with pd.ExcelWriter(output_path, engine='openpyxl') as writer:
    df_karsilastirma.to_excel(writer, sheet_name='Aylik_Skorlar', index=False)
    df_korelasyon.to_excel(writer, sheet_name='Korelasyon_Analizi', index=False)
    if len(df_anlamli) > 0:
        df_anlamli.to_excel(writer, sheet_name='Anlamli_Sonuclar', index=False)
print(f' Sonuçlar kaydedildi: {output_path}')

##  Sonuç

Bu notebook'ta:
1.  7 NLP modeli ile **aylık** kriz skoru hesaplandı
2.  Enflasyon ile **lag korelasyonu** analiz edildi
3.  **Kelime Sayma** yöntemi ile karşılaştırma yapıldı
4.  En iyi tahmin modeli belirlendi